# **cleanning**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import magma
from plotnine import ggplot, aes, geom_point, theme_minimal, labs, theme, element_blank
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import statsmodels.stats.outliers_influence as vif
import statsmodels.stats.diagnostic as ssd

**read data**

In [2]:
nhanes = pd.read_excel("Nhanes_v1.xlsx")

In [3]:
nhanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10351 entries, 0 to 10350
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sampl       10351 non-null  int64  
 1   strata      10351 non-null  int64  
 2   psu         10351 non-null  object 
 3   region      10351 non-null  object 
 4   smsa        10351 non-null  object 
 5   location    10351 non-null  int64  
 6   houssiz     10351 non-null  int64  
 7   sex         10351 non-null  object 
 8   race        10351 non-null  object 
 9   age         10351 non-null  int64  
 10  height      10351 non-null  float64
 11  weight      10351 non-null  float64
 12  bpsystol    10351 non-null  int64  
 13  bpdiast     10351 non-null  int64  
 14  tcresult    10351 non-null  int64  
 15  tgresult    5050 non-null   float64
 16  hdresult    8720 non-null   float64
 17  hgb         10351 non-null  float64
 18  hct         10351 non-null  float64
 19  tibc        10351 non-nul

In [4]:
nhanes.head()

,sampl,strata,psu,region,smsa,location,houssiz,sex,race,age,...,hsizgp,rural,loglead,agegrp,highlead,bmi,highbp,hsiz_all,smsa_all,region_all
0,1400,1,PSU 1,S,"SMSA, not central city",1,4,Male,White,54,...,4,Urban,NaN,50–59,NaN,20.495686,0,4,2,3
1,1401,1,PSU 1,S,"SMSA, not central city",1,6,Female,White,41,...,5,Urban,2.564949,40–49,lead<25,21.022337,0,5,2,3
2,1402,1,PSU 1,S,"SMSA, central city",1,6,Female,Other,21,...,5,Urban,NaN,20–29,NaN,24.973860,0,5,1,3
3,1404,1,PSU 1,S,"SMSA, not central city",1,9,Female,White,63,...,5,Urban,NaN,60–69,NaN,35.728722,1,5,2,3
4,1405,1,PSU 1,S,"SMSA, central city",1,3,Female,White,64,...,3,Urban,2.995732,60–69,lead<25,27.923803,0,3,1,3


**calc na**

In [5]:
(nhanes.isna().sum()/len(nhanes)*100).sort_values(ascending= False)

highlead      52.197855
loglead       52.197855
lead          52.197855
tgresult      51.212443
fhtatk        47.502657
hdresult      15.756932
copper        11.786301
zinc          11.100377
vitaminc       3.651821
albumin        3.236402
corpuscl       0.859820
porphyrn       0.782533
diabetes       0.019322
heartatk       0.019322
hlthstat       0.019322
sampl          0.000000
black          0.000000
female         0.000000
orace          0.000000
hsizgp         0.000000
rural          0.000000
agegrp         0.000000
bmi            0.000000
highbp         0.000000
hsiz_all       0.000000
smsa_all       0.000000
trnsfern       0.000000
sizplace       0.000000
leadwt         0.000000
height         0.000000
psu            0.000000
region         0.000000
smsa           0.000000
location       0.000000
houssiz        0.000000
sex            0.000000
race           0.000000
age            0.000000
weight         0.000000
finalwgt       0.000000
bpsystol       0.000000
bpdiast        0

**drop**

In [6]:
def clean_data(nhanes):
    # Drop column: 'female'
    nhanes = nhanes.drop(columns=['female'])
    # Drop column: 'hsiz_all'
    nhanes = nhanes.drop(columns=['hsiz_all'])
    # Drop column: 'black'
    nhanes = nhanes.drop(columns=['black'])
    # Drop column: 'orace'
    nhanes = nhanes.drop(columns=['orace'])
    # Drop column: 'smsa'
    nhanes = nhanes.drop(columns=['smsa'])
    # Drop column: 'region'
    nhanes = nhanes.drop(columns=['region'])
    # Drop column: 'agegrp'
    nhanes = nhanes.drop(columns=['agegrp'])
    # Drop column: 'highlead'
    nhanes = nhanes.drop(columns=['highlead'])
    # Drop column: 'loglead'
    nhanes = nhanes.drop(columns=['loglead'])
    # Drop column: 'tgresult'
    nhanes = nhanes.drop(columns=['tgresult'])
    # Drop column: 'lead'
    nhanes = nhanes.drop(columns=['lead'])
    # Drop column: 'fhtatk'
    nhanes = nhanes.drop(columns=['fhtatk'])
    return nhanes

dnhanes = clean_data(nhanes.copy())
dnhanes.head()

,sampl,strata,psu,location,houssiz,sex,race,age,height,weight,...,vitaminc,zinc,copper,porphyrn,hsizgp,rural,bmi,highbp,smsa_all,region_all
0,1400,1,PSU 1,1,4,Male,White,54,174.598007,62.480000,...,0.4,104.0,156.0,38.0,4,Urban,20.495686,0,2,3
1,1401,1,PSU 1,1,6,Female,White,41,152.296997,48.759998,...,1.2,111.0,139.0,92.0,5,Urban,21.022337,0,2,3
2,1402,1,PSU 1,1,6,Female,Other,21,164.098007,67.250000,...,1.3,102.0,238.0,48.0,5,Urban,24.973860,0,1,3
3,1404,1,PSU 1,1,9,Female,White,63,162.598007,94.459999,...,1.1,109.0,132.0,62.0,5,Urban,35.728722,1,2,3
4,1405,1,PSU 1,1,3,Female,White,64,163.098007,74.279999,...,1.3,99.0,127.0,44.0,3,Urban,27.923803,0,1,3


**coding**

In [7]:
def clean_data(dnhanes):
    # Replace all instances of "PSU 1" with "1" in column: 'psu'
    dnhanes['psu'] = dnhanes['psu'].str.replace("PSU 1", "0", case=False, regex=False)
    # Replace all instances of "PSU 2" with "2" in column: 'psu'
    dnhanes['psu'] = dnhanes['psu'].str.replace("PSU 2", "1", case=False, regex=False)
    # Replace all instances of "Female" with "1" in column: 'sex'
    dnhanes['sex'] = dnhanes['sex'].str.replace("Female", "0", case=False, regex=False)
    # Replace all instances of "Male" with "2" in column: 'sex'
    dnhanes['sex'] = dnhanes['sex'].str.replace("Male", "1", case=False, regex=False)
    # Replace all instances of "White" with "1" in column: 'race'
    dnhanes['race'] = dnhanes['race'].str.replace("White", "0", case=False, regex=False)
    # Replace all instances of "Black" with "2" in column: 'race'
    dnhanes['race'] = dnhanes['race'].str.replace("Black", "1", case=False, regex=False)
    # Replace all instances of "Other" with "3" in column: 'race'
    dnhanes['race'] = dnhanes['race'].str.replace("Other", "2", case=False, regex=False)
    # Replace all instances of "No heart attack" with "1" in column: 'heartatk'
    dnhanes['heartatk'] = dnhanes['heartatk'].str.replace("No heart attack", "0", case=False, regex=False)
    # Replace all instances of "Had heart attack" with "2" in column: 'heartatk'
    dnhanes['heartatk'] = dnhanes['heartatk'].str.replace("Had heart attack", "1", case=False, regex=False)
    # Replace all instances of "Not diabetic" with "1" in column: 'diabetes'
    dnhanes['diabetes'] = dnhanes['diabetes'].str.replace("Not diabetic", "0", case=False, regex=False)
    # Replace all instances of "Diabetic" with "2" in column: 'diabetes'
    dnhanes['diabetes'] = dnhanes['diabetes'].str.replace("Diabetic", "1", case=False, regex=False)
    # Replace all instances of "Urban" with "1" in column: 'rural'
    dnhanes['rural'] = dnhanes['rural'].str.replace("Urban", "0", case=False, regex=False)
    # Replace all instances of "Rural" with "2" in column: 'rural'
    dnhanes['rural'] = dnhanes['rural'].str.replace("Rural", "1", case=False, regex=False)
    return dnhanes

rnhanes = clean_data(dnhanes.copy())
rnhanes.head()

,sampl,strata,psu,location,houssiz,sex,race,age,height,weight,...,vitaminc,zinc,copper,porphyrn,hsizgp,rural,bmi,highbp,smsa_all,region_all
0,1400,1,0,1,4,1,0,54,174.598007,62.480000,...,0.4,104.0,156.0,38.0,4,0,20.495686,0,2,3
1,1401,1,0,1,6,0,0,41,152.296997,48.759998,...,1.2,111.0,139.0,92.0,5,0,21.022337,0,2,3
2,1402,1,0,1,6,0,2,21,164.098007,67.250000,...,1.3,102.0,238.0,48.0,5,0,24.973860,0,1,3
3,1404,1,0,1,9,0,0,63,162.598007,94.459999,...,1.1,109.0,132.0,62.0,5,0,35.728722,1,2,3
4,1405,1,0,1,3,0,0,64,163.098007,74.279999,...,1.3,99.0,127.0,44.0,3,0,27.923803,0,1,3


 Replace all instances of "PSU 1" with "1" in column: 'psu'

 Replace all instances of "PSU 2" with "2" in column: 'psu'

 Replace all instances of "Female" with "1" in column: 'sex
 
 Replace all instances of "Male" with "2" in column: 'sex'

 Replace all instances of "White" with "1" in column: 'race'

 Replace all instances of "Black" with "2" in column: 'race'

 Replace all instances of "Other" with "3" in column: 'race'

 Replace all instances of "No heart attack" with "1" in column: 'heartatk'

 Replace all instances of "Had heart attack" with "2" in column: 'heartatk'

 Replace all instances of "Not diabetic" with "1" in column: 'diabetes'

 Replace all instances of "Diabetic" with "2" in column: 'diabetes'

 eplace all instances of "Urban" with "1" in column: 'rural'

 Replace all instances of "Rural" with "2" in column: 'rural'

In [8]:
rnhanes

,sampl,strata,psu,location,houssiz,sex,race,age,height,weight,...,vitaminc,zinc,copper,porphyrn,hsizgp,rural,bmi,highbp,smsa_all,region_all
0,1400,1,0,1,4,1,0,54,174.598007,62.480000,...,0.4,104.0,156.0,38.0,4,0,20.495686,0,2,3
1,1401,1,0,1,6,0,0,41,152.296997,48.759998,...,1.2,111.0,139.0,92.0,5,0,21.022337,0,2,3
2,1402,1,0,1,6,0,2,21,164.098007,67.250000,...,1.3,102.0,238.0,48.0,5,0,24.973860,0,1,3
3,1404,1,0,1,9,0,0,63,162.598007,94.459999,...,1.1,109.0,132.0,62.0,5,0,35.728722,1,2,3
4,1405,1,0,1,3,0,0,64,163.098007,74.279999,...,1.3,99.0,127.0,44.0,3,0,27.923803,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10346,48760,32,1,48,5,0,0,35,166.397995,56.360001,...,0.9,87.0,122.0,46.0,5,1,20.355173,0,3,2
10347,48763,32,1,48,2,0,0,33,160.500000,107.279999,...,1.2,114.0,153.0,74.0,2,1,41.645557,1,3,2
10348,48764,32,1,48,1,0,0,60,164.000000,95.820000,...,0.9,90.0,144.0,39.0,1,0,35.626114,0,3,2
10349,48768,32,1,48,1,0,0,29,167.296997,53.750000,...,1.0,67.0,194.0,42.0,1,0,19.204464,0,3,2


**impute**

In [9]:
num_nhanes = rnhanes.select_dtypes(include=['number'])

In [10]:
imputer = IterativeImputer()
fit_imput = imputer.fit_transform(num_nhanes)
imputed_nhanes = pd.DataFrame(fit_imput, columns = num_nhanes.columns)

In [11]:
imputed_nhanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10351 entries, 0 to 10350
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sampl       10351 non-null  float64
 1   strata      10351 non-null  float64
 2   location    10351 non-null  float64
 3   houssiz     10351 non-null  float64
 4   age         10351 non-null  float64
 5   height      10351 non-null  float64
 6   weight      10351 non-null  float64
 7   bpsystol    10351 non-null  float64
 8   bpdiast     10351 non-null  float64
 9   tcresult    10351 non-null  float64
 10  hdresult    10351 non-null  float64
 11  hgb         10351 non-null  float64
 12  hct         10351 non-null  float64
 13  tibc        10351 non-null  float64
 14  iron        10351 non-null  float64
 15  finalwgt    10351 non-null  float64
 16  leadwt      10351 non-null  float64
 17  corpuscl    10351 non-null  float64
 18  trnsfern    10351 non-null  float64
 19  albumin     10351 non-nul

**select object data**

**concat full data**

In [12]:
obj_nahanes =rnhanes.select_dtypes(include=[object])
full_nhanes = pd.concat([imputed_nhanes,obj_nahanes],axis=1)

In [13]:
full_nhanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10351 entries, 0 to 10350
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sampl       10351 non-null  float64
 1   strata      10351 non-null  float64
 2   location    10351 non-null  float64
 3   houssiz     10351 non-null  float64
 4   age         10351 non-null  float64
 5   height      10351 non-null  float64
 6   weight      10351 non-null  float64
 7   bpsystol    10351 non-null  float64
 8   bpdiast     10351 non-null  float64
 9   tcresult    10351 non-null  float64
 10  hdresult    10351 non-null  float64
 11  hgb         10351 non-null  float64
 12  hct         10351 non-null  float64
 13  tibc        10351 non-null  float64
 14  iron        10351 non-null  float64
 15  finalwgt    10351 non-null  float64
 16  leadwt      10351 non-null  float64
 17  corpuscl    10351 non-null  float64
 18  trnsfern    10351 non-null  float64
 19  albumin     10351 non-nul

**show precentage of outliers**

In [14]:
(full_nhanes.isna().sum()/len(full_nhanes)*100).sort_values(ascending= False)

diabetes      0.019322
heartatk      0.019322
hlthstat      0.019322
sampl         0.000000
highbp        0.000000
zinc          0.000000
copper        0.000000
porphyrn      0.000000
hsizgp        0.000000
bmi           0.000000
region_all    0.000000
smsa_all      0.000000
albumin       0.000000
psu           0.000000
sex           0.000000
race          0.000000
sizplace      0.000000
vitaminc      0.000000
trnsfern      0.000000
strata        0.000000
corpuscl      0.000000
location      0.000000
houssiz       0.000000
age           0.000000
height        0.000000
weight        0.000000
bpsystol      0.000000
bpdiast       0.000000
tcresult      0.000000
hdresult      0.000000
hgb           0.000000
hct           0.000000
tibc          0.000000
iron          0.000000
finalwgt      0.000000
leadwt        0.000000
rural         0.000000
dtype: float64

**drop na**

In [15]:
gnhanes=full_nhanes.dropna(axis=0)

In [16]:
(gnhanes.isna().sum()/len(gnhanes)*100).sort_values(ascending= False)

sampl         0.0
albumin       0.0
zinc          0.0
copper        0.0
porphyrn      0.0
hsizgp        0.0
bmi           0.0
highbp        0.0
smsa_all      0.0
region_all    0.0
psu           0.0
sex           0.0
race          0.0
hlthstat      0.0
heartatk      0.0
diabetes      0.0
sizplace      0.0
vitaminc      0.0
trnsfern      0.0
strata        0.0
corpuscl      0.0
location      0.0
houssiz       0.0
age           0.0
height        0.0
weight        0.0
bpsystol      0.0
bpdiast       0.0
tcresult      0.0
hdresult      0.0
hgb           0.0
hct           0.0
tibc          0.0
iron          0.0
finalwgt      0.0
leadwt        0.0
rural         0.0
dtype: float64

In [17]:
gnhanes.head()

,sampl,strata,location,houssiz,age,height,weight,bpsystol,bpdiast,tcresult,...,smsa_all,region_all,psu,sex,race,hlthstat,heartatk,diabetes,sizplace,rural
0,1400.0,1.0,1.0,4.0,54.0,174.598007,62.480000,106.0,80.0,226.0,...,2.0,3.0,0,1,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0
1,1401.0,1.0,1.0,6.0,41.0,152.296997,48.759998,108.0,66.0,179.0,...,2.0,3.0,0,0,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0
2,1402.0,1.0,1.0,6.0,21.0,164.098007,67.250000,98.0,66.0,137.0,...,1.0,3.0,0,0,2,Good,0,0,"Urbanized area; 1,000,000–2,999,999",0
3,1404.0,1.0,1.0,9.0,63.0,162.598007,94.459999,180.0,80.0,189.0,...,2.0,3.0,0,0,0,Fair,0,1,"Urbanized area; 1,000,000–2,999,999",0
4,1405.0,1.0,1.0,3.0,64.0,163.098007,74.279999,120.0,76.0,311.0,...,1.0,3.0,0,0,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0


**change types**

In [18]:
def clean_data(gnhanes):
    # Change column type to int64 for column: 'houssiz'
    gnhanes = gnhanes.astype({'houssiz': 'int64'})
    # Change column type to int64 for column: 'age'
    gnhanes = gnhanes.astype({'age': 'int64'})
    # Change column type to float64 for column: 'height'
    gnhanes = gnhanes.astype({'height': 'float64'})
    # Change column type to float64 for column: 'weight'
    gnhanes = gnhanes.astype({'weight': 'float64'})
    # Change column type to int64 for column: 'bpsystol'
    gnhanes = gnhanes.astype({'bpsystol': 'int64'})
    # Change column type to int64 for column: 'bpdiast'
    gnhanes = gnhanes.astype({'bpdiast': 'int64'})
    # Change column type to float64 for column: 'tcresult'
    gnhanes = gnhanes.astype({'tcresult': 'float64'})
    # Change column type to float64 for column: 'hdresult'
    gnhanes = gnhanes.astype({'hdresult': 'float64'})
    # Change column type to float64 for column: 'hgb'
    gnhanes = gnhanes.astype({'hgb': 'float64'})
    # Change column type to int64 for column: 'psu'
    gnhanes = gnhanes.astype({'psu': 'int64'})
    # Change column type to int64 for column: 'sex'
    gnhanes = gnhanes.astype({'sex': 'int64'})
    # Change column type to int64 for column: 'race'
    gnhanes = gnhanes.astype({'race': 'int64'})
    # Change column type to int64 for column: 'heartatk'
    gnhanes = gnhanes.astype({'heartatk': 'int64'})
    # Change column type to int64 for column: 'diabetes'
    gnhanes = gnhanes.astype({'diabetes': 'int64'})
    # Change column type to int64 for column: 'rural'
    gnhanes = gnhanes.astype({'rural': 'int64'})
    # Change column type to int64 for column: 'region_all'
    gnhanes = gnhanes.astype({'region_all': 'int64'})
    # Change column type to int64 for column: 'smsa_all'
    gnhanes = gnhanes.astype({'smsa_all': 'int64'})
    # Change column type to int64 for column: 'highbp'
    gnhanes = gnhanes.astype({'highbp': 'int64'})
    # Change column type to int64 for column: 'hsizgp'
    gnhanes = gnhanes.astype({'hsizgp': 'int64'})
    return gnhanes

cnhanes = clean_data(gnhanes.copy())
cnhanes.head()

,sampl,strata,location,houssiz,age,height,weight,bpsystol,bpdiast,tcresult,...,smsa_all,region_all,psu,sex,race,hlthstat,heartatk,diabetes,sizplace,rural
0,1400.0,1.0,1.0,4,54,174.598007,62.480000,106,80,226.0,...,2,3,0,1,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0
1,1401.0,1.0,1.0,6,41,152.296997,48.759998,108,66,179.0,...,2,3,0,0,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0
2,1402.0,1.0,1.0,6,21,164.098007,67.250000,98,66,137.0,...,1,3,0,0,2,Good,0,0,"Urbanized area; 1,000,000–2,999,999",0
3,1404.0,1.0,1.0,9,63,162.598007,94.459999,180,80,189.0,...,2,3,0,0,0,Fair,0,1,"Urbanized area; 1,000,000–2,999,999",0
4,1405.0,1.0,1.0,3,64,163.098007,74.279999,120,76,311.0,...,1,3,0,0,0,Very good,0,0,"Urbanized area; 1,000,000–2,999,999",0
